In [1]:
!pip -q install -U "openai>=1.54.0" datasets tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.0/812.0 kB 11.2 MB/s eta 0:00:00


In [2]:
import os
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"  
REASONING_EFFORT = "low"            

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)

print("Client ready with OpenAI-compatible Gemini endpoint.")


Client ready with OpenAI-compatible Gemini endpoint.


In [3]:
# # Load the dataset


from datasets import load_dataset

ds = load_dataset("mohammad-shirkhani/social_movielens_custom", split="train")
print(ds)
print("Rows:", len(ds))
print("Columns:", ds.column_names)

# Peek first row
ex0 = ds[0]
print("\nFirst row keys:", list(ex0.keys()))

# Pretty-short preview
for k, v in ex0.items():
    if isinstance(v, list) and len(v) > 3:
        print(f"- {k}: list(len={len(v)}), first 2:\n  {v[:2]}")
    else:
        print(f"- {k}: {v}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/691 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/29.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Dataset({
    features: ['user', 'item', 'answer', 'paths'],
    num_rows: 20000
})
Rows: 20000
Columns: ['user', 'item', 'answer', 'paths']

First row keys: ['user', 'item', 'answer', 'paths']
- user: {'UserID': 911, 'Age': 37, 'Gender': 'Female', 'Occupation': 'writer'}
- item: {'MovieID': 193, 'Title': 'Right Stuff, The (1983)', 'ReleaseDate': '01-Jan-1983', 'Genres': 'Drama'}
- answer: 4.0
- paths: list(len=20), first 2:
  ['user_question -> user_item_2 (rating=2) -> Item{MovieID 98, Title "Silence of the Lambs, The (1991)", Release Date 01-Jan-1991, Genres Drama, Thriller} -> item_user_4 (rating=4) -> User{UserID 498, Age 26, Gender Male, Occupation writer} -> usersim -> User{UserID 58, Age 27, Gender Male, Occupation programmer} -> user_item_3 (rating=3) -> item_question', 'user_question -> usersim -> User{UserID 716, Age 36, Gender Female, Occupation administrator} -> user_item_5 (rating=5) -> Item{MovieID 517, Title "Manhattan (1979)", Release Date 01-Jan-1979, Genres Comedy, D

In [4]:
from typing import Dict, Any

def detect_fields(example: Dict[str, Any]):
    user_keys = ["user", "User", "user_dict", "UserDict"]
    item_keys = ["item", "Item", "item_dict", "ItemDict"]
    rating_keys = ["answer", "rating", "label", "score"]
    paths_keys = ["paths", "meta_paths", "metapaths", "path_list", "path", "metaPaths"]

    def find_key(candidates, predicate=None):
        for k in candidates:
            if k in example and (predicate(example[k]) if predicate else True):
                return k
        for k, v in example.items():
            if predicate and predicate(v):
                return k
        return None

    user_key = find_key(user_keys, predicate=lambda v: isinstance(v, dict))
    item_key = find_key(item_keys, predicate=lambda v: isinstance(v, dict))
    rating_key = find_key(rating_keys, predicate=lambda v: isinstance(v, (float, int)))
    paths_key = find_key(paths_keys, predicate=lambda v: isinstance(v, list))

    if user_key is None:
        user_key = next((k for k, v in example.items() if isinstance(v, dict)), None)
    if item_key is None:
        cand = [k for k, v in example.items() if isinstance(v, dict) and k != user_key]
        item_key = cand[0] if cand else None
    if rating_key is None:
        cand = [k for k, v in example.items() if isinstance(v, (float, int))]
        rating_key = cand[0] if cand else None
    if paths_key is None:
        cand = [k for k, v in example.items() if isinstance(v, list)]
        paths_key = cand[0] if cand else None

    return user_key, item_key, rating_key, paths_key

user_key, item_key, rating_key, paths_key = detect_fields(ex0)
print("Detected keys ->",
      "user:", user_key,
      "| item:", item_key,
      "| rating:", rating_key,
      "| paths:", paths_key)

# Extract first example pieces for later
user0 = ex0[user_key]
item0 = ex0[item_key]
rating0 = ex0[rating_key]
paths0 = ex0[paths_key]
print("\nUser example:", user0)
print("Item example:", item0)
print("Rating example:", rating0)
print("Meta-paths:", len(paths0))

Detected keys -> user: user | item: item | rating: answer | paths: paths

User example: {'UserID': 911, 'Age': 37, 'Gender': 'Female', 'Occupation': 'writer'}
Item example: {'MovieID': 193, 'Title': 'Right Stuff, The (1983)', 'ReleaseDate': '01-Jan-1983', 'Genres': 'Drama'}
Rating example: 4.0
Meta-paths: 20


In [5]:
# Output must ONLY contain <reason>...</reason> then <answer>...</answer>.

from typing import List

def format_kv_block(title: str, d: Dict[str, Any]) -> str:
    lines = [f"{title}:"]
    for k, v in d.items():
        lines.append(f"- {k}: {v}")
    return "\n".join(lines)

def format_meta_paths(paths: List[str]) -> str:
    lines = ["Meta-path evidence (each path from this user to the target item):"]
    for p in paths:
        p = p.strip()
        if not p.startswith("- "):
            p = "- " + p
        lines.append(p)
    return "\n".join(lines)

def build_prompt_evidence_then_answer(
    user: Dict[str, Any],
    item: Dict[str, Any],
    rating: float,
    meta_paths: List[str],
) -> str:
    user_block = format_kv_block("User", user)
    item_block = format_kv_block("Item", item)
    observed_rating_str = str(rating)
    meta_block = format_meta_paths(meta_paths)

    instruction = (
        "Task: Extract evidence from the provided data and then conclude the numerical rating.\n\n"
        "You are given a heterogeneous bipartite graph setting (users and items). Edges include:\n"
        "- user→item rating interactions (e.g., user_item_k with an explicit rating),\n"
        "- usersim (user-user similarity), and\n"
        "- itemsim (item-item similarity).\n\n"
        "What to do:\n"
        "1) Analyze the user's likely preferences and the item's traits by leveraging ONLY:\n"
        "   - the user attributes,\n"
        "   - the item attributes, and\n"
        "   - the provided meta-paths (treat each path as a weak but interpretable signal; combine corroborating signals).\n"
        "   - the provided meta-paths (treat each path as a weak but interpretable signal; combine corroborating signals).\n"
        "2) Then produce two XML blocks ONLY (no extra text):\n"
        "   a) <reason>...</reason> — Provide a clear, evidence-first explanation that states\n"
        "      what information you obtain from the user/item/meta-paths and how those signals combine.\n"
        "      Avoid wording like “because the rating is X”. Instead, present evidence → inference, and end with a neutral\n"
        "      sentence such as: “Therefore, the rating equals {R}.”\n"
        "   b) <answer>{R}</answer> — Put the observed rating number {R} exactly as given below.\n\n"
        "STRICT FORMAT RULES:\n"
        "- Output ONLY these two blocks in this order, nothing else:\n"
        "<reason>\n"
        "...your evidence-first explanation here...\n"
        "</reason>\n"
        "<answer>{R}</answer>\n"
        "- Do not invent attributes, paths, or ratings not present in the input. If something is unknown, treat it as unknown.\n"
    )

    observed_block = f"Observed rating (user → item): {observed_rating_str}"
    prompt = (
        instruction
        + "\n=== INPUT ===\n"
        + user_block + "\n\n"
        + item_block + "\n\n"
        + observed_block + "\n\n"
        + meta_block + "\n"
        + "=== END INPUT ===\n"
    )
    return prompt.replace("{R}", observed_rating_str)

prompt0 = build_prompt_evidence_then_answer(user0, item0, rating0, paths0)
print("Prompt:\n")
print(prompt0[:])


Prompt:

Task: Extract evidence from the provided data and then conclude the numerical rating.

You are given a heterogeneous bipartite graph setting (users and items). Edges include:
- user→item rating interactions (e.g., user_item_k with an explicit rating),
- usersim (user-user similarity), and
- itemsim (item-item similarity).

What to do:
1) Analyze the user's likely preferences and the item's traits by leveraging ONLY:
   - the user attributes,
   - the item attributes, and
   - the provided meta-paths (treat each path as a weak but interpretable signal; combine corroborating signals).
   - the provided meta-paths (treat each path as a weak but interpretable signal; combine corroborating signals).
2) Then produce two XML blocks ONLY (no extra text):
   a) <reason>...</reason> — Provide a clear, evidence-first explanation that states
      what information you obtain from the user/item/meta-paths and how those signals combine.
      Avoid wording like “because the rating is X”. In

In [6]:
# # Single example run with Gemini 2.5 Flash-Lite (OpenAI style)

import re

def call_gemini(prompt: str, max_tokens: int = 2048):
    resp = client.chat.completions.create(
        model=MODEL_ID,
        reasoning_effort=REASONING_EFFORT,  # "low"
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        n=1,
    )
    # 'content' is the text we need
    return resp.choices[0].message.content

def parse_reason_answer(text: str):
    reason = None
    answer = None
    m_reason = re.search(r"<reason>(.*?)</reason>", text, flags=re.DOTALL | re.IGNORECASE)
    if m_reason:
        reason = m_reason.group(1).strip()
    m_answer = re.search(r"<answer>\s*([0-9]+(?:\.[0-9]+)?)\s*</answer>", text, flags=re.DOTALL | re.IGNORECASE)
    if m_answer:
        answer = m_answer.group(1).strip()
    return reason, answer

In [7]:
# Rate limiting helpers

import time
import math
from collections import deque

RPM_LIMIT = 15           
TPM_LIMIT = 250_000     
OUT_TOKENS_BUDGET = 2048 

class RateLimiter:
    def __init__(self, rpm: int, tpm: int):
        self.rpm = rpm
        self.tpm = tpm
        self.req_times = deque()
        self.token_times = deque()  

    @staticmethod
    def estimate_tokens(text: str) -> int:
        return max(1, math.ceil(len(text) / 4))

    def wait(self, prompt_text: str, out_tokens:int = OUT_TOKENS_BUDGET):
        now = time.time()
        while self.req_times and now - self.req_times[0] > 60:
            self.req_times.popleft()
        while self.token_times and now - self.token_times[0][0] > 60:
            self.token_times.popleft()

        in_tokens = self.estimate_tokens(prompt_text)
        while len(self.req_times) >= self.rpm:
            sleep_s = 60 - (now - self.req_times[0])
            time.sleep(max(0.01, sleep_s))
            now = time.time()
            while self.req_times and now - self.req_times[0] > 60:
                self.req_times.popleft()

        used_tpm = sum(t for (_, t) in self.token_times)
        while used_tpm + in_tokens + out_tokens > self.tpm:
            sleep_s = 60 - (now - self.token_times[0][0])
            time.sleep(max(0.01, sleep_s))
            now = time.time()
            while self.token_times and now - self.token_times[0][0] > 60:
                self.token_times.popleft()
            used_tpm = sum(t for (_, t) in self.token_times)

        self.req_times.append(time.time())
        self.token_times.append((time.time(), in_tokens + out_tokens))

limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

def run_one_example(example, max_tokens:int=2048):
    user = example[user_key]
    item = example[item_key]
    rating = example[rating_key]
    meta = example[paths_key]

    prompt = build_prompt_evidence_then_answer(user, item, rating, meta)

    limiter.wait(prompt, out_tokens=max_tokens)

    backoff = 1.0
    for attempt in range(5):
        try:
            text = call_gemini(prompt, max_tokens=max_tokens)
            reason, answer = parse_reason_answer(text)
            return text, reason, answer
        except Exception as e:
            if attempt == 4:
                raise
            time.sleep(backoff)
            backoff *= 2.0


In [8]:
import os, json
from tqdm.auto import tqdm
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"
REASONING_EFFORT = "low"

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)
print("Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).")

START_INDEX = 5900
END_INDEX   = 6800   
OUT_PATH    = "/content/gemini_flash_lite_reason_answer_5900_6799.jsonl"

ok = 0
missing_reason = 0
missing_answer = 0
failures = 0


limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

with open(OUT_PATH, "w", encoding="utf-8") as f:
    pbar = tqdm(range(START_INDEX, END_INDEX),
                desc="Generating explanations (rows 5900..6799, Gemini 2.5 Flash-Lite, reasoning_effort=low)")
    for i in pbar:
        try:
            ex = ds[i]
            content, reason, answer = run_one_example(ex, max_tokens=2048)

            rec = {
                "index": i,
                "user": ex[user_key],
                "item": ex[item_key],
                "rating": float(ex[rating_key]),
                "reason": reason,
                "answer": answer
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

            if reason is None:
                missing_reason += 1
            if answer is None:
                missing_answer += 1
            if (reason is not None) and (answer is not None):
                ok += 1

        except Exception as e:
            failures += 1

        pbar.set_postfix({
            "ok": ok,
            "miss_reason": missing_reason,
            "miss_answer": missing_answer,
            "fail": failures
        })

print("\nDone (rows 5900..6799).")
print(f"OK: {ok} | Missing reason: {missing_reason} | Missing answer: {missing_answer} | Failures: {failures}")
print(f"Saved to: {OUT_PATH}")

# Quick peek of first 3 lines
try:
    with open(OUT_PATH, "r", encoding="utf-8") as f:
        print("\nSample outputs:")
        for _ in range(3):
            line = next(f).strip()
            print(json.loads(line))
except Exception as e:
    print("Preview error:", e)


Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).


Generating explanations (rows 5900..6799, Gemini 2.5 Flash-Lite, reasoning_effort=low):   0%|          | 0/900…


Done (rows 5900..6799).
OK: 880 | Missing reason: 20 | Missing answer: 20 | Failures: 0
Saved to: /content/gemini_flash_lite_reason_answer_5900_6799.jsonl

Sample outputs:
{'index': 5900, 'user': {'UserID': 921, 'Age': 20, 'Gender': 'Female', 'Occupation': 'student'}, 'item': {'MovieID': 190, 'Title': 'Henry V (1989)', 'ReleaseDate': '01-Jan-1989', 'Genres': 'Drama, War'}, 'rating': 2.0, 'reason': 'The user, UserID 921, is a 20-year-old female student. The target item, MovieID 190, "Henry V (1989)", is classified under the genres Drama and War. Analyzing the meta-paths, we observe several indirect signals:\n\n1.  **User 921\'s implicit preferences via similar users:** User 921 is linked through `usersim` to UserID 747 (Age 19, Male, student). User 747 has a `user_item_4` interaction (rating 4) with MovieID 692 ("American President, The"), which includes the "Drama" genre. While this is a positive rating for a drama, it\'s not directly for Item 190.\n\n2.  **Item 190\'s genre associati

In [9]:
import os, json
from tqdm.auto import tqdm
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"
REASONING_EFFORT = "low"

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)
print("Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).")

START_INDEX = 6800
END_INDEX   = 7700   
OUT_PATH    = "/content/gemini_flash_lite_reason_answer_6800_7699.jsonl"

ok = 0
missing_reason = 0
missing_answer = 0
failures = 0


limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

with open(OUT_PATH, "w", encoding="utf-8") as f:
    pbar = tqdm(range(START_INDEX, END_INDEX),
                desc="Generating explanations (rows 6800..7699, Gemini 2.5 Flash-Lite, reasoning_effort=low)")
    for i in pbar:
        try:
            ex = ds[i]
            content, reason, answer = run_one_example(ex, max_tokens=2048)

            rec = {
                "index": i,
                "user": ex[user_key],
                "item": ex[item_key],
                "rating": float(ex[rating_key]),
                "reason": reason,
                "answer": answer
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

            if reason is None:
                missing_reason += 1
            if answer is None:
                missing_answer += 1
            if (reason is not None) and (answer is not None):
                ok += 1

        except Exception as e:
            failures += 1

        pbar.set_postfix({
            "ok": ok,
            "miss_reason": missing_reason,
            "miss_answer": missing_answer,
            "fail": failures
        })

print("\nDone (rows 6800..7699).")
print(f"OK: {ok} | Missing reason: {missing_reason} | Missing answer: {missing_answer} | Failures: {failures}")
print(f"Saved to: {OUT_PATH}")

# Quick peek of first 3 lines
try:
    with open(OUT_PATH, "r", encoding="utf-8") as f:
        print("\nSample outputs:")
        for _ in range(3):
            line = next(f).strip()
            print(json.loads(line))
except Exception as e:
    print("Preview error:", e)


Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).


Generating explanations (rows 6800..7699, Gemini 2.5 Flash-Lite, reasoning_effort=low):   0%|          | 0/900…


Done (rows 6800..7699).
OK: 825 | Missing reason: 16 | Missing answer: 16 | Failures: 59
Saved to: /content/gemini_flash_lite_reason_answer_6800_7699.jsonl

Sample outputs:
{'index': 6800, 'user': {'UserID': 626, 'Age': 23, 'Gender': 'Male', 'Occupation': 'scientist'}, 'item': {'MovieID': 328, 'Title': 'Conspiracy Theory (1997)', 'ReleaseDate': '08-Aug-1997', 'Genres': 'Action, Mystery, Romance, Thriller'}, 'rating': 1.0, 'reason': 'The user is 23 years old, male, and works as a scientist. The target item, MovieID 328, is "Conspiracy Theory (1997)" with genres Action, Mystery, Romance, and Thriller.\nThe provided meta-paths detail connections between a "user_question" and an "item_question" through various user and item nodes and their interactions/similarities. However, none of the provided meta-paths directly link UserID 626 to MovieID 328, nor do they contain any user nodes with UserID 626 or item nodes with MovieID 328. Consequently, no specific evidence from the meta-paths can be

In [8]:
import os, json
from tqdm.auto import tqdm
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"
REASONING_EFFORT = "low"

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)
print("Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).")

START_INDEX = 7700
END_INDEX   = 8600   
OUT_PATH    = "/content/gemini_flash_lite_reason_answer_7700_8599.jsonl"

ok = 0
missing_reason = 0
missing_answer = 0
failures = 0

limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

with open(OUT_PATH, "w", encoding="utf-8") as f:
    pbar = tqdm(range(START_INDEX, END_INDEX),
                desc="Generating explanations (rows 7700..8599, Gemini 2.5 Flash-Lite, reasoning_effort=low)")
    for i in pbar:
        try:
            ex = ds[i]
            content, reason, answer = run_one_example(ex, max_tokens=2048)

            rec = {
                "index": i,
                "user": ex[user_key],
                "item": ex[item_key],
                "rating": float(ex[rating_key]),
                "reason": reason,
                "answer": answer
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

            if reason is None:
                missing_reason += 1
            if answer is None:
                missing_answer += 1
            if (reason is not None) and (answer is not None):
                ok += 1

        except Exception as e:
            failures += 1

        pbar.set_postfix({
            "ok": ok,
            "miss_reason": missing_reason,
            "miss_answer": missing_answer,
            "fail": failures
        })

print("\nDone (rows 7700..8599).")
print(f"OK: {ok} | Missing reason: {missing_reason} | Missing answer: {missing_answer} | Failures: {failures}")
print(f"Saved to: {OUT_PATH}")

# Quick peek of first 3 lines
try:
    with open(OUT_PATH, "r", encoding="utf-8") as f:
        print("\nSample outputs:")
        for _ in range(3):
            line = next(f).strip()
            print(json.loads(line))
except Exception as e:
    print("Preview error:", e)


Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).


Generating explanations (rows 7700..8599, Gemini 2.5 Flash-Lite, reasoning_effort=low):   0%|          | 0/900…


Done (rows 7700..8599).
OK: 882 | Missing reason: 18 | Missing answer: 18 | Failures: 0
Saved to: /content/gemini_flash_lite_reason_answer_7700_8599.jsonl

Sample outputs:
{'index': 7700, 'user': {'UserID': 13, 'Age': 47, 'Gender': 'Male', 'Occupation': 'educator'}, 'item': {'MovieID': 193, 'Title': 'Right Stuff, The (1983)', 'ReleaseDate': '01-Jan-1983', 'Genres': 'Drama'}, 'rating': 5.0, 'reason': 'The user is identified as a 47-year-old male educator. The target item is the movie "Right Stuff, The (1983)" which is categorized under the Drama genre. Analysis of the provided meta-paths reveals several signals that align with a high rating.\n\nOne significant path indicates a user rating of 5 for "Shawshank Redemption, The (1994)", a Drama film, and shows item similarity to "The Wizard of Oz, The (1939)", which also includes Drama as a genre. Another path shows a user rating of 5 for "Primary Colors (1998)", also a Drama, connected to another user with a rating of 4. A path involving 

In [9]:
import os, json
from tqdm.auto import tqdm
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"
REASONING_EFFORT = "low"

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)
print("Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).")

START_INDEX = 8600
END_INDEX   = 9500   
OUT_PATH    = "/content/gemini_flash_lite_reason_answer_8600_9499.jsonl"

ok = 0
missing_reason = 0
missing_answer = 0
failures = 0

limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

with open(OUT_PATH, "w", encoding="utf-8") as f:
    pbar = tqdm(range(START_INDEX, END_INDEX),
                desc="Generating explanations (rows 8600..9499, Gemini 2.5 Flash-Lite, reasoning_effort=low)")
    for i in pbar:
        try:
            ex = ds[i]
            content, reason, answer = run_one_example(ex, max_tokens=2048)

            rec = {
                "index": i,
                "user": ex[user_key],
                "item": ex[item_key],
                "rating": float(ex[rating_key]),
                "reason": reason,
                "answer": answer
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

            if reason is None:
                missing_reason += 1
            if answer is None:
                missing_answer += 1
            if (reason is not None) and (answer is not None):
                ok += 1

        except Exception as e:
            failures += 1

        pbar.set_postfix({
            "ok": ok,
            "miss_reason": missing_reason,
            "miss_answer": missing_answer,
            "fail": failures
        })

print("\nDone (rows 8600..9499).")
print(f"OK: {ok} | Missing reason: {missing_reason} | Missing answer: {missing_answer} | Failures: {failures}")
print(f"Saved to: {OUT_PATH}")

# Quick peek of first 3 lines
try:
    with open(OUT_PATH, "r", encoding="utf-8") as f:
        print("\nSample outputs:")
        for _ in range(3):
            line = next(f).strip()
            print(json.loads(line))
except Exception as e:
    print("Preview error:", e)


Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).


Generating explanations (rows 8600..9499, Gemini 2.5 Flash-Lite, reasoning_effort=low):   0%|          | 0/900…


Done (rows 8600..9499).
OK: 877 | Missing reason: 22 | Missing answer: 22 | Failures: 1
Saved to: /content/gemini_flash_lite_reason_answer_8600_9499.jsonl

Sample outputs:
{'index': 8600, 'user': {'UserID': 686, 'Age': 32, 'Gender': 'Male', 'Occupation': 'educator'}, 'item': {'MovieID': 28, 'Title': 'Apollo 13 (1995)', 'ReleaseDate': '01-Jan-1995', 'Genres': 'Action, Drama, Thriller'}, 'rating': 4.0, 'reason': "The user is identified as a 32-year-old male educator. The target item, Apollo 13 (1995), is a movie categorized under Action, Drama, and Thriller genres. The provided meta-paths describe complex, indirect relationships between a 'user_question' and an 'item_question' through sequences of user-user similarity, user-item ratings, and item-item similarity, involving various intermediate users and items with their own attributes and ratings. However, none of the provided meta-paths directly link UserID 686 or MovieID 28. Therefore, the meta-paths do not offer specific evidence abo

In [10]:
import os, json
from tqdm.auto import tqdm
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"
REASONING_EFFORT = "low"

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)
print("Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).")

START_INDEX = 9500
END_INDEX   = 10400   
OUT_PATH    = "/content/gemini_flash_lite_reason_answer_9500_10399.jsonl"

ok = 0
missing_reason = 0
missing_answer = 0
failures = 0

limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

with open(OUT_PATH, "w", encoding="utf-8") as f:
    pbar = tqdm(range(START_INDEX, END_INDEX),
                desc="Generating explanations (rows 9500..10399, Gemini 2.5 Flash-Lite, reasoning_effort=low)")
    for i in pbar:
        try:
            ex = ds[i]
            content, reason, answer = run_one_example(ex, max_tokens=2048)

            rec = {
                "index": i,
                "user": ex[user_key],
                "item": ex[item_key],
                "rating": float(ex[rating_key]),
                "reason": reason,
                "answer": answer
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

            if reason is None:
                missing_reason += 1
            if answer is None:
                missing_answer += 1
            if (reason is not None) and (answer is not None):
                ok += 1

        except Exception as e:
            failures += 1

        pbar.set_postfix({
            "ok": ok,
            "miss_reason": missing_reason,
            "miss_answer": missing_answer,
            "fail": failures
        })

print("\nDone (rows 9500..10399).")
print(f"OK: {ok} | Missing reason: {missing_reason} | Missing answer: {missing_answer} | Failures: {failures}")
print(f"Saved to: {OUT_PATH}")

# Quick peek of first 3 lines
try:
    with open(OUT_PATH, "r", encoding="utf-8") as f:
        print("\nSample outputs:")
        for _ in range(3):
            line = next(f).strip()
            print(json.loads(line))
except Exception as e:
    print("Preview error:", e)


Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).


Generating explanations (rows 9500..10399, Gemini 2.5 Flash-Lite, reasoning_effort=low):   0%|          | 0/90…


Done (rows 9500..10399).
OK: 887 | Missing reason: 13 | Missing answer: 13 | Failures: 0
Saved to: /content/gemini_flash_lite_reason_answer_9500_10399.jsonl

Sample outputs:
{'index': 9500, 'user': {'UserID': 479, 'Age': 30, 'Gender': 'Male', 'Occupation': 'educator'}, 'item': {'MovieID': 108, 'Title': 'Kids in the Hall: Brain Candy (1996)', 'ReleaseDate': '12-Apr-1996', 'Genres': 'Comedy'}, 'rating': 4.0, 'reason': 'The target user, UserID 479, is an educator aged 30, and the target item, MovieID 108, is a Comedy titled "Kids in the Hall: Brain Candy (1996)".\nExamining the meta-paths reveals several indirect signals:\n1.  User similarities are mentioned (e.g., `usersim -> User{UserID 85, Age 51, Gender Male, Occupation educator}`). User 479 shares the "educator" occupation with User 85.\n2.  Several other users with the "educator" occupation appear in paths: UserID 85 is linked via `usersim` and then to items with ratings 2.0 and 2.0. User 479\'s occupation is \'educator\'.\n3.  The

In [11]:
import os, json
from tqdm.auto import tqdm
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"
REASONING_EFFORT = "low"

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)
print("Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).")

START_INDEX = 10400
END_INDEX   = 11300   
OUT_PATH    = "/content/gemini_flash_lite_reason_answer_10400_11299.jsonl"

ok = 0
missing_reason = 0
missing_answer = 0
failures = 0

limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

with open(OUT_PATH, "w", encoding="utf-8") as f:
    pbar = tqdm(range(START_INDEX, END_INDEX),
                desc="Generating explanations (rows 10400..11299, Gemini 2.5 Flash-Lite, reasoning_effort=low)")
    for i in pbar:
        try:
            ex = ds[i]
            content, reason, answer = run_one_example(ex, max_tokens=2048)

            rec = {
                "index": i,
                "user": ex[user_key],
                "item": ex[item_key],
                "rating": float(ex[rating_key]),
                "reason": reason,
                "answer": answer
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

            if reason is None:
                missing_reason += 1
            if answer is None:
                missing_answer += 1
            if (reason is not None) and (answer is not None):
                ok += 1

        except Exception as e:
            failures += 1

        pbar.set_postfix({
            "ok": ok,
            "miss_reason": missing_reason,
            "miss_answer": missing_answer,
            "fail": failures
        })

print("\nDone (rows 10400..11299).")
print(f"OK: {ok} | Missing reason: {missing_reason} | Missing answer: {missing_answer} | Failures: {failures}")
print(f"Saved to: {OUT_PATH}")

# Quick peek of first 3 lines
try:
    with open(OUT_PATH, "r", encoding="utf-8") as f:
        print("\nSample outputs:")
        for _ in range(3):
            line = next(f).strip()
            print(json.loads(line))
except Exception as e:
    print("Preview error:", e)


Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).


Generating explanations (rows 10400..11299, Gemini 2.5 Flash-Lite, reasoning_effort=low):   0%|          | 0/9…


Done (rows 10400..11299).
OK: 886 | Missing reason: 12 | Missing answer: 14 | Failures: 0
Saved to: /content/gemini_flash_lite_reason_answer_10400_11299.jsonl

Sample outputs:
{'index': 10400, 'user': {'UserID': 595, 'Age': 25, 'Gender': 'Male', 'Occupation': 'programmer'}, 'item': {'MovieID': 845, 'Title': 'That Thing You Do! (1996)', 'ReleaseDate': '28-Sep-1996', 'Genres': 'Comedy'}, 'rating': 3.0, 'reason': 'The user, aged 25 and a programmer, has an observed rating of 3.0 for the item "That Thing You Do!" (1996), categorized under the \'Comedy\' genre. Meta-path analysis reveals several indirect connections and preference signals. One meta-path links to Item 151 ("Willy Wonka and the Chocolate Factory"), which shares the \'Comedy\' genre and received a rating of 5 from UserID 664 (Age 30, Male, engineer), suggesting a positive reception for comedy films within a potential network. Another path points to Item 111 ("Truth About Cats & Dogs"), also tagged \'Comedy\' and \'Romance\', 

In [12]:
import os, json
from tqdm.auto import tqdm
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"
REASONING_EFFORT = "low"

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)
print("Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).")

START_INDEX = 11300
END_INDEX   = 12200   
OUT_PATH    = "/content/gemini_flash_lite_reason_answer_11300_12199.jsonl"

ok = 0
missing_reason = 0
missing_answer = 0
failures = 0


limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

with open(OUT_PATH, "w", encoding="utf-8") as f:
    pbar = tqdm(range(START_INDEX, END_INDEX),
                desc="Generating explanations (rows 11300..12199, Gemini 2.5 Flash-Lite, reasoning_effort=low)")
    for i in pbar:
        try:
            ex = ds[i]
            content, reason, answer = run_one_example(ex, max_tokens=2048)

            rec = {
                "index": i,
                "user": ex[user_key],
                "item": ex[item_key],
                "rating": float(ex[rating_key]),
                "reason": reason,
                "answer": answer
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

            if reason is None:
                missing_reason += 1
            if answer is None:
                missing_answer += 1
            if (reason is not None) and (answer is not None):
                ok += 1

        except Exception as e:
            failures += 1

        pbar.set_postfix({
            "ok": ok,
            "miss_reason": missing_reason,
            "miss_answer": missing_answer,
            "fail": failures
        })

print("\nDone (rows 11300..12199).")
print(f"OK: {ok} | Missing reason: {missing_reason} | Missing answer: {missing_answer} | Failures: {failures}")
print(f"Saved to: {OUT_PATH}")

# Quick peek of first 3 lines
try:
    with open(OUT_PATH, "r", encoding="utf-8") as f:
        print("\nSample outputs:")
        for _ in range(3):
            line = next(f).strip()
            print(json.loads(line))
except Exception as e:
    print("Preview error:", e)


Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).


Generating explanations (rows 11300..12199, Gemini 2.5 Flash-Lite, reasoning_effort=low):   0%|          | 0/9…


Done (rows 11300..12199).
OK: 883 | Missing reason: 17 | Missing answer: 17 | Failures: 0
Saved to: /content/gemini_flash_lite_reason_answer_11300_12199.jsonl

Sample outputs:
{'index': 11300, 'user': {'UserID': 782, 'Age': 21, 'Gender': 'Female', 'Occupation': 'artist'}, 'item': {'MovieID': 880, 'Title': 'Soul Food (1997)', 'ReleaseDate': '01-Jan-1997', 'Genres': 'Drama'}, 'rating': 4.0, 'reason': 'The user (UserID 782) is a 21-year-old female artist. The target item (MovieID 880) is "Soul Food (1997)", a Drama film released in January 1997.\n\nSeveral meta-paths provide weak signals that can be combined:\n1.  A path via `usersim` connects User 782 to User 827 (Age 23, Female, engineer), who interacted with "Wag the Dog (1997)" (Genres: Comedy, Drama) with a rating of 3. This indicates a potential inclination towards Drama content through similar users.\n2.  Another path via `usersim` connects User 782 to User 787 (Age 18, Female, student), who interacted with "Devil\'s Advocate, The

In [13]:
import os, json
from tqdm.auto import tqdm
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"
REASONING_EFFORT = "low"

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)
print("Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).")

START_INDEX = 12200
END_INDEX   = 13100   
OUT_PATH    = "/content/gemini_flash_lite_reason_answer_12200_13099.jsonl"

ok = 0
missing_reason = 0
missing_answer = 0
failures = 0

limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

with open(OUT_PATH, "w", encoding="utf-8") as f:
    pbar = tqdm(range(START_INDEX, END_INDEX),
                desc="Generating explanations (rows 12200..13099, Gemini 2.5 Flash-Lite, reasoning_effort=low)")
    for i in pbar:
        try:
            ex = ds[i]
            content, reason, answer = run_one_example(ex, max_tokens=2048)

            rec = {
                "index": i,
                "user": ex[user_key],
                "item": ex[item_key],
                "rating": float(ex[rating_key]),
                "reason": reason,
                "answer": answer
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

            if reason is None:
                missing_reason += 1
            if answer is None:
                missing_answer += 1
            if (reason is not None) and (answer is not None):
                ok += 1

        except Exception as e:
            failures += 1

        pbar.set_postfix({
            "ok": ok,
            "miss_reason": missing_reason,
            "miss_answer": missing_answer,
            "fail": failures
        })

print("\nDone (rows 12200..13099).")
print(f"OK: {ok} | Missing reason: {missing_reason} | Missing answer: {missing_answer} | Failures: {failures}")
print(f"Saved to: {OUT_PATH}")

# Quick peek of first 3 lines
try:
    with open(OUT_PATH, "r", encoding="utf-8") as f:
        print("\nSample outputs:")
        for _ in range(3):
            line = next(f).strip()
            print(json.loads(line))
except Exception as e:
    print("Preview error:", e)


Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).


Generating explanations (rows 12200..13099, Gemini 2.5 Flash-Lite, reasoning_effort=low):   0%|          | 0/9…


Done (rows 12200..13099).
OK: 883 | Missing reason: 16 | Missing answer: 17 | Failures: 0
Saved to: /content/gemini_flash_lite_reason_answer_12200_13099.jsonl

Sample outputs:
{'index': 12200, 'user': {'UserID': 627, 'Age': 24, 'Gender': 'Male', 'Occupation': 'engineer'}, 'item': {'MovieID': 226, 'Title': 'Die Hard 2 (1990)', 'ReleaseDate': '01-Jan-1990', 'Genres': 'Action, Thriller'}, 'rating': 1.0, 'reason': 'The user, UserID 627, is a 24-year-old male engineer. The target item, MovieID 226, is an Action and Thriller movie released in 1990.\n\nSeveral meta-paths provide weak signals suggesting a potential mismatch or neutral sentiment, which aligns with the observed low rating of 1.0.\n\n1.  **Paths involving dissimilar items and genres:**\n    *   One path connects the user (via `user_item_4`, rating 4) to "Dead Man Walking" (Drama), which then links via `item_user_1` (rating 1) to UserID 648 (Male, 43, engineer). This path shows the user interacting with a Drama genre, and this co

In [15]:
import os, json
from tqdm.auto import tqdm
from openai import OpenAI

os.environ["GEMINI_API_KEY"] = ""

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_ID = "gemini-2.5-flash-lite"
REASONING_EFFORT = "low"

client = OpenAI(
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=BASE_URL
)
print("Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).")

START_INDEX = 13100
END_INDEX   = 14000   
OUT_PATH    = "/content/gemini_flash_lite_reason_answer_13100_13999.jsonl"

ok = 0
missing_reason = 0
missing_answer = 0
failures = 0

limiter = RateLimiter(RPM_LIMIT, TPM_LIMIT)

with open(OUT_PATH, "w", encoding="utf-8") as f:
    pbar = tqdm(range(START_INDEX, END_INDEX),
                desc="Generating explanations (rows 13100..13999, Gemini 2.5 Flash-Lite, reasoning_effort=low)")
    for i in pbar:
        try:
            ex = ds[i]
            content, reason, answer = run_one_example(ex, max_tokens=2048)

            rec = {
                "index": i,
                "user": ex[user_key],
                "item": ex[item_key],
                "rating": float(ex[rating_key]),
                "reason": reason,
                "answer": answer
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

            if reason is None:
                missing_reason += 1
            if answer is None:
                missing_answer += 1
            if (reason is not None) and (answer is not None):
                ok += 1

        except Exception as e:
            failures += 1

        pbar.set_postfix({
            "ok": ok,
            "miss_reason": missing_reason,
            "miss_answer": missing_answer,
            "fail": failures
        })

print("\nDone (rows 13100..13999).")
print(f"OK: {ok} | Missing reason: {missing_reason} | Missing answer: {missing_answer} | Failures: {failures}")
print(f"Saved to: {OUT_PATH}")

# Quick peek of first 3 lines
try:
    with open(OUT_PATH, "r", encoding="utf-8") as f:
        print("\nSample outputs:")
        for _ in range(3):
            line = next(f).strip()
            print(json.loads(line))
except Exception as e:
    print("Preview error:", e)


Client reconfigured with NEW API key (OpenAI-compatible Gemini endpoint).


Generating explanations (rows 13100..13999, Gemini 2.5 Flash-Lite, reasoning_effort=low):   0%|          | 0/9…


Done (rows 13100..13999).
OK: 881 | Missing reason: 19 | Missing answer: 19 | Failures: 0
Saved to: /content/gemini_flash_lite_reason_answer_13100_13999.jsonl

Sample outputs:
{'index': 13100, 'user': {'UserID': 670, 'Age': 30, 'Gender': 'Male', 'Occupation': 'technician'}, 'item': {'MovieID': 651, 'Title': 'Glory (1989)', 'ReleaseDate': '01-Jan-1989', 'Genres': 'Action, Drama, War'}, 'rating': 4.0, 'reason': 'The target user (UserID 670) is a 30-year-old male technician. The target item (MovieID 651) is "Glory (1989)", a War, Action, and Drama film released in 1989. Several meta-paths suggest positive preferences related to the target item\'s genres and release year, and potentially similar user profiles.\n\nSpecifically, one meta-path links the user through an intermediate item, "Full Metal Jacket (1987)" (Action, Drama, War), via an `itemsim` connection to "Glory (1989)". This suggests a similarity between "Full Metal Jacket" and "Glory" in terms of their War and Drama genres, whic